In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import string
from statistics import mean

In [2]:
import numpy as np
import pandas as pd

zomato = pd.read_csv("C:/Users/hp/Downloads/archive/zomato.csv")
zomato.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [3]:
pwd

'C:\\Users\\hp'

In [4]:
zomato.describe()


,votes
count,51717.000000
mean,283.697527
std,803.838853
min,0.000000
25%,7.000000
50%,41.000000
75%,198.000000
max,16832.000000


In [5]:
zomato.columns


Index(['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines',
       'approx_cost(for two people)', 'reviews_list', 'menu_item',
       'listed_in(type)', 'listed_in(city)'],
      dtype='object')

In [6]:
zomato2 = zomato.drop(['url','dish_liked','phone'], axis=1)


In [7]:
print(zomato2.duplicated().sum())


43


In [8]:
zomato2.drop_duplicates(inplace=True)


In [9]:
print(zomato2.isnull().sum())


address                           0
name                              0
online_order                      0
book_table                        0
rate                           7767
votes                             0
location                         21
rest_type                       227
cuisines                         45
approx_cost(for two people)     345
reviews_list                      0
menu_item                         0
listed_in(type)                   0
listed_in(city)                   0
dtype: int64


In [10]:
zomato2.dropna(how='any', inplace=True)


In [11]:
zomato = zomato2.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type', 'listed_in(city)':'city'})


In [12]:
zomato['cost'] = zomato['cost'].astype(str) #Changing the cost to string
zomato['cost'] = zomato['cost'].apply(lambda x: x.replace(',','.')) #Using lambda function to replace ',' from cost
zomato['cost'] = zomato['cost'].astype(float)

In [13]:
zomato.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [14]:

zomato = zomato.loc[zomato.rate !='NEW']
zomato = zomato.loc[zomato.rate !='-'].reset_index(drop=True)
remove_slash = lambda x: x.replace('/5', '') if type(x) == np.str else x
zomato.rate = zomato.rate.apply(remove_slash).str.strip().astype('float')

In [15]:
zomato.name = zomato.name.apply(lambda x:x.title())
zomato.online_order.replace(('Yes','No'),(True, False),inplace=True)
zomato.book_table.replace(('Yes','No'),(True, False),inplace=True)

In [16]:
zomato.name

0                                                    Jalsa
1                                           Spice Elephant
2                                          San Churro Cafe
3                                    Addhuri Udupi Bhojana
4                                            Grand Village
                               ...                        
41232                           The Farm House Bar N Grill
41233                                              Bhagini
41234    Best Brews - Four Points By Sheraton Bengaluru...
41235    Chime - Sheraton Grand Bengaluru Whitefield Ho...
41236                         The Nest - The Den Bengaluru
Name: name, Length: 41237, dtype: object

In [17]:
zomato.book_table

0         True
1        False
2        False
3        False
4        False
         ...  
41232    False
41233    False
41234    False
41235     True
41236    False
Name: book_table, Length: 41237, dtype: bool

In [18]:
restaurants = list(zomato['name'].unique())
zomato['Mean Rating'] = 0

for i in range(len(restaurants)):
    zomato['Mean Rating'][zomato['name'] == restaurants[i]] = zomato['rate'][zomato['name'] == restaurants[i]].mean()

In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (1,5))
zomato[['Mean Rating']] = scaler.fit_transform(zomato[['Mean Rating']]).round(2)

In [20]:
zomato["reviews_list"] = zomato["reviews_list"].str.lower()


In [21]:
zomato.reviews_list

0        [('rated 4.0', 'rated\n  a beautiful place to ...
1        [('rated 4.0', 'rated\n  had been here for din...
2        [('rated 3.0', "rated\n  ambience is not that ...
3        [('rated 4.0', "rated\n  great food and proper...
4        [('rated 4.0', 'rated\n  very good restaurant ...
                               ...                        
41232    [('rated 4.0', 'rated\n  ambience- big and spa...
41233    [('rated 4.0', 'rated\n  a fine place to chill...
41234    [('rated 5.0', "rated\n  food and service are ...
41235    [('rated 4.0', 'rated\n  nice and friendly pla...
41236    [('rated 5.0', 'rated\n  great ambience , look...
Name: reviews_list, Length: 41237, dtype: object

In [22]:
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_punctuation(text))

In [23]:
PUNCT_TO_REMOVE

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [24]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])
zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_stopwords(text))

In [25]:
import re
def removeurls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: removeurls(text))

zomato[['reviews_list', 'cuisines']].sample(10)

,reviews_list,cuisines
37183,rated 40 ratedn nice place visit mantri mall f...,"Fast Food, Beverages"
39996,rated 20 ratedn ordered special chicken biryan...,North Indian
8978,rated 30 ratedn actual rating 28nvery average ...,Continental
6427,rated 40 ratedn amazing food love soups ordere...,"European, French, Salad"
12252,rated 40 ratedn biryani lover right placeits c...,Biryani
40631,rated 40 ratedn small cosy place around good d...,"Ice Cream, Cafe, Pizza, Burger, Desserts, Beve..."
20612,rated 40 ratedn went sunday nightnnambience ni...,South Indian
27376,rated 40 ratedn ambience place really cozy nic...,"Cafe, Beverages"
12482,rated 50 ratedn went first time due gold privi...,"Cafe, Continental, Beverages, Desserts"
34815,rated 40 ratedn went breakfast saturday ordere...,South Indian


In [26]:
restaurant_names = list(zomato['name'].unique())
def get_top_words(column, top_nu_of_words, nu_of_word):
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    bag_of_words = vec.fit_transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:top_nu_of_words]
    
zomato=zomato.drop(['address','rest_type', 'type', 'menu_item', 'votes'],axis=1)
import pandas

# Randomly sample 60% of your dataframe
df_percent = zomato.sample(frac=0.5)


In [27]:
df_percent = zomato.sample(frac=0.5)

df_percent.set_index('name', inplace=True)
indices = pd.Series(df_percent.index)

In [28]:
# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [29]:
def recommend(name, cosine_similarities = cosine_similarities):
    
    # Create a list to put top restaurants
    recommend_restaurant = []
    
    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]
    
    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:31].index)
    
    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])
    
    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    
    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','Mean Rating', 'cost']][df_percent.index == each].sample()))
    
    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    
    return df_new


TOP 10 RESTAURANTS LIKE Pai Vihar WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Burma Burma,"Asian, Burmese",4.74,1.5
Skoolroom,"Cafe, Continental, Italian, Burger, Beverages",4.23,700.0
Andhra Ruchulu,"Andhra, North Indian",3.72,800.0
Andhra Ruchulu,South Indian,3.72,250.0
Dum Biryani Hub,Biryani,3.71,700.0
Green Pepper,"Seafood, South Indian, Chinese, Kerala",3.65,600.0
1992 Chats - Space,Street Food,3.45,200.0
3 Leafs,"North Indian, South Indian, Chinese",3.45,600.0
Chicken Corner,"Biryani, North Indian, Fast Food, Chinese",3.35,400.0
Queen Pearls,Pizza,3.32,500.0


In [30]:
recommend('3 Leafs')

TOP 10 RESTAURANTS LIKE 3 Leafs WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Burma Burma,"Asian, Burmese",4.74,1.5
Chianti,Italian,4.59,1.5
Skoolroom,"Cafe, Continental, Italian, Burger, Beverages",4.23,700.0
Church Street Social,"American, North Indian, Chinese, Finger Food, ...",4.23,1.5
Mojo Pizza - 2X Toppings,Pizza,4.13,600.0
Nando'S,"Portuguese, Wraps, Burger, Salad",4.13,1.2
Marzipan Cafe & Bakery,"Cafe, Mediterranean, Bakery, Greek, Beverages",4.10,700.0
The Higher Taste,"Pizza, Biryani, North Indian, South Indian",3.90,800.0
Mani'S Dum Biryani,"Biryani, Andhra",3.77,750.0
Andhra Ruchulu,"Andhra, North Indian",3.72,800.0


In [31]:
recommend('Jalsa')

TOP 10 RESTAURANTS LIKE Jalsa WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Asia Kitchen By Mainland China,"Asian, Chinese, Thai, Momos",5.00,1.5
Biergarten,"Continental, European, BBQ, Chinese, Asian",4.83,2.4
Biergarten,"Continental, North Indian, Chinese, European, ...",4.83,2.1
The Black Pearl,"North Indian, European, Mediterranean, BBQ",4.78,1.5
The Black Pearl,"North Indian, European, Mediterranean",4.78,1.4
Big Pitcher,"American, Continental, North Indian, Mediterra...",4.68,1.8
Communiti,"Continental, BBQ, Salad",4.67,1.5
The Pallet,"Continental, Mediterranean, Italian, North Ind...",4.48,1.6
Jalsa Gold,"North Indian, Mughlai, Italian",4.48,1.3
Brooks And Bonds Brewery,"Continental, Mediterranean, North Indian, Chin...",4.45,1.6


In [32]:
recommend('The Black Pearl')

TOP 10 RESTAURANTS LIKE The Black Pearl WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Biergarten,"Continental, North Indian, Chinese, European, ...",4.83,2.1
Big Pitcher,"American, Continental, North Indian, Mediterra...",4.68,1.8
Hammered,"North Indian, South Indian, Continental, Asian",4.65,1.2
Hammered,"North Indian, Thai, Japanese, Continental, Cafe",4.65,1.3
Simon Says Brew Works,"Continental, American, Italian, Modern Indian,...",4.48,1.4
Barbeque Nation,"North Indian, European, Mediterranean, BBQ, Kebab",4.47,1.6
Brooks And Bonds Brewery,"Continental, Mediterranean, North Indian, Chin...",4.45,1.6
Delhi Highway,North Indian,4.41,1.2
The Rig,"Continental, Finger Food, Pizza",4.23,1.5
Barbecoa,"North Indian, Continental, Chettinad, Andhra, ...",4.23,1.0


In [33]:
recommend('Skoolroom')

TOP 5 RESTAURANTS LIKE Skoolroom WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Communiti,"Continental, BBQ, Salad",4.67,1.5
Dice N Dine,"Continental, Cafe, Italian, Fast Food, Steak",4.23,900.0
Smally'S Resto Cafe,"Cafe, Italian, Burger, American, Steak",4.06,650.0
Eggzotic,"North Indian, Chinese, Biryani, Fast Food",3.77,500.0
Cinnamon,"North Indian, Chinese, Biryani",3.62,550.0


In [34]:
recommend('Barbecoa')

TOP 10 RESTAURANTS LIKE Barbecoa WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Asia Kitchen By Mainland China,"Asian, Chinese, Thai, Momos",5.00,1.5
Byg Brewski Brewing Company,"Continental, North Indian, Italian, South Indi...",5.00,1.6
Ab'S - Absolute Barbecues,"European, Mediterranean, North Indian, BBQ",4.86,1.6
Big Pitcher,"American, Continental, North Indian, Mediterra...",4.68,1.8
Hammered,"North Indian, Thai, Japanese, Continental, Cafe",4.65,1.3
Jalsa Gold,"North Indian, Mughlai, Italian",4.48,1.3
Brooks And Bonds Brewery,"Continental, Mediterranean, North Indian, Chin...",4.45,1.6
Delhi Highway,"North Indian, Mughlai",4.41,1.5
Foxtrot,"North Indian, Chinese, Continental, Momos",4.35,1.2
The Fisherman'S Wharf,"Seafood, Goan, North Indian, Continental, Asian",4.30,1.4


In [35]:
recommend('Foxtrot')

TOP 10 RESTAURANTS LIKE Foxtrot WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Burma Burma,"Asian, Burmese",4.74,1.5
Arbor Brewing Company,"American, Continental",4.48,2.0
Brooks And Bonds Brewery,"Continental, Mediterranean, North Indian, Chin...",4.45,1.6
Oh! Calcutta,"Bengali, Seafood",4.39,1.2
CafãÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ© Felix,"American, Cafe, Continental",4.35,1.7
Nando'S,"Portuguese, African",4.13,1.2
Mojo Pizza - 2X Toppings,Pizza,4.13,600.0
Marzipan Cafe & Bakery,"Cafe, Mediterranean, Bakery, Greek",4.10,700.0
The Higher Taste,"Pizza, Biryani, North Indian, South Indian",3.90,800.0
Dum Biryani Hub,Biryani,3.71,700.0


In [36]:
recommend('Oh! Calcutta')

TOP 3 RESTAURANTS LIKE Oh! Calcutta WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Esplanade,Bengali,4.23,1.0
Cinnamon,"North Indian, Chinese, Biryani",3.62,550.0
Old Stories,"Bengali, North Indian, Oriya",3.45,600.0
